# Valores nulos en SQL
### **Ingeniería de datos**
**Profesor: Domagoj Vrgoč**

### Introducción

En esta clase continuaremos con nuestro análisis de la base de datos de actores y peliculas. Esta vey revisaremos que ocurre al tener valores nulos en nuestra base de datos.

### Requisitos

Para esta actividad, así como en todas las actividades de SQL, vamos a utilizar *Google colab* (https://colab.research.google.com), que es un entorno virtual permitiendo armar un servidor de bases de datos, y conectarse con este servidor. Para la conexión ocuparemos la herramienta llamada Jupyter Notebooks. Esta herramienta permite conectarse con un servidor SQL de la misma manera cómo hacerlo a través de la consola en un servidor local. 

El motor de bases de datos que ocuparemos en este curso se llama PostgreSQL, y uno siempre puede instalarlo localmente en su computador. Idea de ocupar Google colab es saltarse este paso, y no tener problemas con instalar, habilitar, o correr un motor de bases de datos.

Por lo tanto, para una actividad de SQL, en este curso siempre ocuparemos Jupyter Notebooks con Google colab. Para esto, se les entregará un archivo con extensión .ipynb, cual hay que subir a la plataforma Google Colab. Al inicio del tutorial mostraremos cómo funciona este proceso.


### Esquema

Para esta actividad, de nuevo trabajaremos con el siguiente esquema:

- `Peliculas(pid, pnombre, paño, pcategoria, pcalificacion, pdirector)`

- `Actores(aid, anombre, aedad)`

- `actuo_en(aid, pid, rol)`

Este esquema corresponde a películas, actores, y la información de los roles interpretados por una actor en una película.

Las llaves en nuestro caso son:
1. `pid`, para `Peliculas`
2. `aid` para `Actores`
3. `(aid,pid,rol)` para `actuo_en`.

Es importante notar que en este caso `rol` forma parte de la llave primaria de la relación `actuo_en`, dado que debemos modelar el caso donde un actor juega dos roles distintos en una misma película.

## Tutorial

Lo primero que hay que hacer es subir este notebook a https://colab.research.google.com

### Iniciar el servidor

Para iniciar el servidor virtual, *instalar* la base de datos postgres debe correr el siguiente bloque:

In [ ]:
# install
!apt update
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"
# set connection
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres

### Creando la base de datos

Para crear y poblar nuestra base de datos, corramo el siguiente bloque de código:

In [ ]:
%%sql

DROP TABLE IF EXISTS Peliculas;
DROP TABLE IF EXISTS Actores;
DROP TABLE IF EXISTS Actuo_En;

CREATE TABLE IF NOT EXISTS Peliculas(
    pid int PRIMARY KEY,
    pnombre varchar(30),
    paño int,
    pcategoria varchar(30),
    pcalificacion float,
    pdirector varchar(30)
);

CREATE TABLE Actores(
    aid int PRIMARY KEY,
    anombre varchar(30),
    aedad int
);

CREATE TABLE actuo_en(
    aid int,
    pid int,
    rol varchar(30),
    PRIMARY KEY (aid,pid,rol)
);

INSERT INTO Peliculas VALUES(1,'Avengers:Endgame',2019,'SciFi',8.4,'Brothers Russo');
INSERT INTO Peliculas VALUES(2,'Captain America: Civil War',2016,'SciFi',NULL,'Brothers Russo');
INSERT INTO Peliculas VALUES(3,'Iron Man',2008,'SciFi',9.1,'John Favreu');
INSERT INTO Peliculas VALUES(4,'Batman: The Dark Knight',2008,'Thriller',8.3,'Christoper Nolan');
INSERT INTO Peliculas VALUES(5,'Batman: The Dark Knight Rises',2012,'Thriller',NULL,'Christoper Nolan');
INSERT INTO Peliculas VALUES(6,'Interstellar',2014,'Drama',8.4,'Christoper Nolan');
INSERT INTO Peliculas VALUES(7,'Sherlock Holmes',2009,'Mystery',7.8,'Guy Ritchie');
INSERT INTO Peliculas VALUES(8,'Avengers: Age of Ultron',NULL,'SciFi',8.3,'Joss Whedon');
INSERT INTO Peliculas VALUES(9,'Doctor Strange',2016,'SciFi',8.8,'Scott Derrickson');

INSERT INTO Actores VALUES(1,'Robert Downey Jr.',57);
INSERT INTO Actores VALUES(2,'Scarlett Johansson',37);
INSERT INTO Actores VALUES(3,'Chris Evans',NULL);
INSERT INTO Actores VALUES(4,'Christian Bale',48);
INSERT INTO Actores VALUES(5,'Anne Hathaway',NULL);
INSERT INTO Actores VALUES(6,'Paul Bettany',50);
INSERT INTO Actores VALUES(7,'Benedict Cumberbatch',45);

INSERT INTO actuo_en VALUES(1,1,'Tony Stark');
INSERT INTO actuo_en VALUES(1,2,'Tony Stark');
INSERT INTO actuo_en VALUES(1,3,'Tony Stark');
INSERT INTO actuo_en VALUES(1,7,'Sherlock Holmes');
INSERT INTO actuo_en VALUES(2,1,'Natasha Romanoff');
INSERT INTO actuo_en VALUES(2,2,'Natasha Romanoff');
INSERT INTO actuo_en VALUES(3,1,'Steve Rogers');
INSERT INTO actuo_en VALUES(3,2,'Steve Rogers');
INSERT INTO actuo_en VALUES(4,4,'Bruce Wayne');
INSERT INTO actuo_en VALUES(4,5,'Bruce Wayne');
INSERT INTO actuo_en VALUES(5,5,'Selina Kyle');
INSERT INTO actuo_en VALUES(5,6,'Amelia Brand');
INSERT INTO actuo_en VALUES(6,8,'J.A.R.V.I.S.');
INSERT INTO actuo_en VALUES(6,8,'Vision');
INSERT INTO actuo_en VALUES(7,9,'Doctor Strange');
INSERT INTO actuo_en VALUES(7,9,'Dormammu');
INSERT INTO actuo_en VALUES(1,8,'Tony Stark');
INSERT INTO actuo_en VALUES(3,8,'Steve Rogers');
INSERT INTO actuo_en VALUES(2,8,'Natasha Romanoff');
INSERT INTO actuo_en VALUES(6,3,'J.A.R.V.I.S.');


Ahora podemos visualizar el contenido de las tablas:

In [ ]:
%%sql

SELECT * FROM Peliculas;

In [ ]:
%%sql

SELECT * FROM Actores;

In [ ]:
%%sql

SELECT * FROM actuo_en;

### Analizando la base de datos con consultas anidadas

Como podemos ver, ahora nuestra base de datos cuenta con algunos valores nulos. En este turorial ilustraremos que pasa con nuestras consultas en este caso, y ilustraremos unos comportamientos inesperados que pueden ocurrir.

### Pregunta 1

empezaremos analizando las películas. Primero, nos interesaría recuperar todas las películas de la categoría `SciFi` con la calificación mayor a mínima califiación entre todas las películas. Intuitivamente, uno espera que con esto recuperaremos todas las películas `SciFi`:

In [ ]:
%%sql

SELECT pnombre
FROM Peliculas
WHERE pcategoria = 'SciFi' AND 
      pcalificacion >= (
                        SELECT MIN(P2.pcalificacion)
                        FROM Peliculas AS P2
                        )


Cómo podemos observar, la película `Captain America: Civil War` no aparece en el listado. Esto ocurre por dos razones:
1. Porque la subconsulta calculando la mínima calificación ignora a los nulos, y devuelve el número 7.8.
2. Al hacer la comparación con pcalificacion y el mínimo devuelto por la consulta, los valores nulos hacen el resultado `unknown`.

### Pregunta 2

Ocupando subconsultas puede resultar en comportamiento incluso más extraño en la presencia de los nulos. Por ejemplo, intentemos ahora encontrar los actores cuya edad es menor que la edad de cualquier actor que aparece en una película `Thriller`. Para esto ocuparemos consultas anidadas:

In [ ]:
%%sql

SELECT A1.anombre
FROM Actores AS A1
WHERE A1.aedad < ALL (
                     SELECT A2.aedad
                     FROM ACTORES AS A2, Peliculas, Actuo_En
                     WHERE A2.aid = Actuo_En.aid AND 
                           Actuo_En.pid = Peliculas.pid AND
                           Peliculas.pcategoria = 'Thriller'
                     ) 

Aquí la consulta anidada simplemente recupera las edades de todos los actores que aparecen en una película `SciFi`, y la consulta exterior compara la edad de actor con esta lista de valores. El problema es que el actor `Chris Evans` tiene valor `NULL` cómo su edad, y aparece en una película `SciFi`. Por lo tanto, al comparar la edad de cualquier actor con este valor nulo, no podemos conseguir true cómo el resultado, y por lo tanto no se devuelve ningun resultado.

Para excluir los nulos desde esta comparación, podemos modificar nuestra consulta de la siguiente manera:

In [ ]:
%%sql

SELECT A1.anombre
FROM Actores AS A1
WHERE A1.aedad < ALL (
                     SELECT A2.aedad
                     FROM ACTORES AS A2, Peliculas, Actuo_En
                     WHERE A2.aid = Actuo_En.aid AND 
                           Actuo_En.pid = Peliculas.pid AND
                           Peliculas.pcategoria = 'Thriller' AND
                           A2.aedad IS NOT NULL
                     ) 

Alternativamente, lo mismo podemos realizar con el operador `MIN`, dado que este ignora a los valores nulos:

In [ ]:
%%sql

SELECT A1.anombre
FROM Actores AS A1
WHERE A1.aedad < (
                     SELECT MIN(A2.aedad)
                     FROM ACTORES AS A2, Peliculas, Actuo_En
                     WHERE A2.aid = Actuo_En.aid AND 
                           Actuo_En.pid = Peliculas.pid AND
                           Peliculas.pcategoria = 'Thriller'
                 ) 

### Pregunta 3

Ahora exploraremos que ocurre cuando intentamos calcular la sma de edades de todos los actores en cada película. Esto podemos realizar de la siguiente forma:

In [ ]:
%%sql

SELECT Peliculas.pid, Peliculas.pnombre, SUM(Actores.aedad)
FROM Peliculas, Actores, actuo_en
WHERE Peliculas.pid = actuo_en.pid AND actuo_en.aid = Actores.aid
GROUP BY Peliculas.pid, Peliculas.pnombre
ORDER BY Peliculas.pid;

Aquí podemos observar que el operador `SUM` simplemente ignora a los valores nulos. La única exepción es la película `Interstellar`, dónde solo tenemos una actriz registrada para esta película, y su edad es un valor `NULL`.

Pese que esto puede aparecer un comportamiento razonable, ignorar campos con nulos puede resultar en comportamiento inesperado. Por ejemplo, si decidimos computar el promedio de edad de actores por película a mano (sumando sus edades, y dividienvo con en número de actores), podemos recibir distinto resultado que ocupando la funcción `AVG`.

Ilustremos esto ahora:


In [ ]:
%%sql

SELECT Peliculas.pid, Peliculas.pnombre, SUM(Actores.aedad), COUNT(DISTINCT Actores.aid), SUM(Actores.aedad)/COUNT(DISTINCT Actores.aid)
FROM Peliculas, Actores, actuo_en
WHERE Peliculas.pid = actuo_en.pid AND actuo_en.aid = Actores.aid
GROUP BY Peliculas.pid, Peliculas.pnombre
ORDER BY Peliculas.pid;

Observen que aquí si contamos los actores sin la edad especificada. Por otro lado, ocupando el operador `AVG` recibiremos:

In [ ]:
%%sql

SELECT Peliculas.pid, Peliculas.pnombre, AVG(Actores.aedad)
FROM Peliculas, Actores, actuo_en
WHERE Peliculas.pid = actuo_en.pid AND actuo_en.aid = Actores.aid
GROUP BY Peliculas.pid, Peliculas.pnombre
ORDER BY Peliculas.pid;

La diferencia aquí es que al ignorar la fila con el valor nulo, por ejemplo en la película `Avengers:Endgame`, el operador AVG divide la suma de edades no nulos con 2 y no con 3, qué es el número total de los actores en esta película.

### Pregunta 4

Pese a comportamiento extraño que pueden causar, nulos son útiles en muchos escenarios. Uno de estos es preservación de la infromación. Considere ahora que a nuestra tabla `Peliculas` insertamos la siguiente tupla:

In [ ]:
%%sql

INSERT INTO Peliculas VALUES(10,'Morbius',2022,'SciFi',NULL,'Daniel Espinosa');

SELECT *
FROM Peliculas;

Para la nueva película que insertamos todvía no tenemos actores ingresados en la tabla `Actuo_En`.

Considere ahora el escenario dónde nos gustaría generar una tabla que contiene cada película, junto con los id de los actores que actuan en ella Intuitivamente, esto podemos hacer con la siguiente consulta:

In [ ]:
%%sql

SELECT Peliculas.pnombre, actuo_en.aid
FROM Peliculas, actuo_en
WHERE Peliculas.pid = actuo_en.pid
ORDER BY Peliculas.pnombre, actuo_en.aid;

Aquí podemos observar que ya no contamos con la información de la película `Morbius`. El problema aquí es que al hacer el join entre `Peliculas` y `Actuo_En`, no podemos realizar un join con la película `Morbius`, dado que no tenemos actores registrados para esta película.

En el caso que queremos todavía disponer de esta información, podemos ocupar un join especial que se llama `LEFT OUTTER JOIN`. Efectivamente, este join nos dice que cuando uno puede realizar el join lo realiza, pero cuando una tupla de la relación de la izquierda del join no puede hacer join con ninguna tupla de la relación de la derecha, los campos faltantes se llenan con nulos. Noten que cuando una tupla sí hace el join, esto no ocurre. En nuestro ejemplo podemos ocupar:

In [ ]:
%%sql

SELECT Peliculas.pnombre, actuo_en.aid
FROM Peliculas LEFT OUTER JOIN actuo_en ON Peliculas.pid = actuo_en.pid
ORDER BY Peliculas.pnombre, actuo_en.aid;

Cómo podemos observar, la película `Morbius` ahora si aparece en nuestro resultado.

En general, la sintáxis de left outter join es:

`Tabla1 LEFT OUTER JOIN Tabla 2 ON Tabla1.atr1 = Tabla2.atr2, Tabla1.atr3 = Tabla2.atr4, ... `

Aquí `Tabla1` es la tabla del lado izquierdo, `Tabla2` la tabla de la derecha, y al poder hacer join de una tupla de `Tabla1` con una tupla de `Tabla2`, este resultado se retorna. Si `Tabla1` contiene una tupla que no puede hacer join con ninguna tupla de la `Tabla2`, los valores que faltan para el join se llennan con nulos.

En SQL existen dos tipos de outter joins adicionales:
- `RIGHT OUTTER JOIN`, que cambia el rol de las relaciones del lado izquierdo y del lado derecho; y
- `FULL OUTTER JOIN`, que combina los `LEFT y RIGHT OUTTER JOIN`.

Para aprender más sobre outter joins, se sugiere consultar el libro del curso, o el siguiente link: https://en.wikipedia.org/wiki/Join_(SQL)#Outer_join

### Resumen

En este tutorial revisamos las cosas que pueden ocurrir al ocupar valores nulos en una base de datos. Adicionalmente, también explicamos el concepto de OUTTER JOINS, que son una forma de hacer joins que se aprovecha de la presencia de valores nulos.